In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
import time
import matplotlib.pyplot as plt

def get_interpolated_depth_vector(img):
    def find_nearest_min(array, value):
        idx = (np.abs(array - value)).argmin()
        return idx

    def find_nearest_max(array, value):
        idx = (np.abs(array - value)).argmax()
        return idx

    def get_zero_patches(idx_zero):

        def split(arr, cond):
            return [arr[cond], arr[~cond]]

        diff_vector = np.zeros(1+idx_zero.shape[0])
        diff_vector[1:]= idx_zero
        differences = np.subtract(idx_zero, diff_vector[0:-1])[1:]
        indices = np.where(differences>1)
        #print(indices, differences)
        zero_patches = []

        if indices[0].size != 0:
            #extract first zero patch
            zero_patches.append(idx_zero[np.where(idx_zero <= idx_zero[indices[0][0]])])
            #extract the following zero patches
            for i in range(indices[0][:].shape[0],0,-1):
                zero_patches.append(idx_zero[np.where(idx_zero > idx_zero[indices[0][i-1]])])
        if indices[0].size == 0:
            zero_patches.append(idx_zero)
        return(zero_patches)

    depth_vector = np.zeros(img.shape[1])
    for i in range(0,img.shape[1]):
        layer = np.argwhere(img[:,i])
        if layer.size != 0:  
            depth_vector[i] = max(layer) - min(layer)
    #interpolation of missing deoth information
    #get all indices
    idx_nonzero = np.argwhere(depth_vector)
    idx_zero = np.where(depth_vector==0)[0]
    #check if list is empty = no zero patches
    if len(idx_zero) != 0:
        #get list with seperate zero patches
        zero_patches = get_zero_patches(idx_zero)
        #print(zero_patches)
        #find interpolation value       
        for patch in zero_patches:
            #print(patch)
            closest_min = find_nearest_min(idx_nonzero, min(patch))
            closest_max = find_nearest_min(idx_nonzero, max(patch)) 
            #print(closest_min, closest_max)
            #print(depth_vector[idx_nonzero[closest_min]],depth_vector[idx_nonzero[closest_max]])
            interpolation = (depth_vector[idx_nonzero[closest_min]]+depth_vector[idx_nonzero[closest_max]])/2
            #print(interpolation)
            #interpolate
            depth_vector[patch] = interpolation

    return depth_vector


In [4]:
data_dir = "/home/olle/PycharmProjects/derive_depth_vector_and_map/data/Volume/OCT"
lat = os.listdir(data_dir)
for late in lat:
    lat_dir = os.path.join(data_dir, late)
    times = os.listdir(lat_dir)
    for t in times:
        time_path = os.path.join(lat_dir, t)
        images = os.listdir(time_path)
        for image in images:
            if "seg" in image and "DV" not in image:
                start = time.time()
                image_path = os.path.join(time_path, image)
                img = cv2.imread(image_path,0)
                depth_vector = get_interpolated_depth_vector(img)
                print(depth_vector.shape)
                np.save(time_path +"/DV_"+image, depth_vector)
                end = time.time()
                print(end - start)

(512,)
0.0653998851776
(512,)
0.0445609092712
(512,)
0.0464630126953
(512,)
0.046275138855
(512,)
0.0482242107391
(512,)
0.0518610477448
(512,)
0.0462889671326
(512,)
0.0453219413757
(512,)
0.0439698696136
(512,)
0.0471630096436
(512,)
0.0423510074615
(512,)
0.0392820835114
(512,)
0.0384058952332
(512,)
0.0408780574799
(512,)
0.0378031730652
(512,)
0.0374710559845
(512,)
0.0501301288605
(512,)
0.0402119159698
(512,)
0.0406489372253
(512,)
0.0461430549622
(512,)
0.0428721904755
(512,)
0.0424270629883
(512,)
0.0533769130707
(512,)
0.0457789897919
(512,)
0.0467219352722
(512,)
0.04656291008
(512,)
0.0520780086517
(512,)
0.0460901260376
(512,)
0.0458869934082
(512,)
0.0437219142914
(512,)
0.041426897049
(512,)
0.0429999828339
(512,)
0.0440828800201
(512,)
0.042111158371
(512,)
0.0462901592255
(512,)
0.0477278232574
(512,)
0.0495419502258
(512,)
0.0492398738861
(512,)
0.0443429946899
(512,)
0.0435521602631
(512,)
0.0433061122894
(512,)
0.0547859668732
(512,)
0.0475161075592
(512,)
0.0405559